In [5]:
import os, sys
import numpy as np
mass=5000
channel="WW"
# channel="numunumu"

profile='NFW'
mcinj="1122_disCSMSx-3"
mcfit="1122"
process='ann'
Ntrials = 2000
Njobs = 5
SignalSub = True


In [2]:

cfig_file = '/home/tchau/code/condor_submit/TSdist/dagman.config'
with open(cfig_file, 'w') as c:
    c.write('DAGMAN_MAX_JOBS_SUBMITTED=500'+'\n')
    c.write('DAGMAN_MAX_SUBMIT_PER_INTERVAL=20'+'\n')
    c.write('DAGMAN_USER_LOG_SCAN_INTERVAL=10'+'\n')
    c.close()
if SignalSub:
    sub_file = '/home/tchau/code/condor_submit/TSdist/TSdist_SignalSubLLH.submit'
else:
    sub_file = '/home/tchau/code/condor_submit/TSdist/TSdist_NormalLLH.submit'

with open(sub_file, 'w') as s:
    s.write('executable = /data/user/tchau/Sandbox/GC_OscNext/Sensitivity/TS_distribution.py'+'\n')
    s.write('\n')
    s.write('initialdir = /data/user/tchau/'+'\n')
    s.write('logdir = /scratch/tchau/DM_OscNext/MakePDF/logs/'+'\n')
    s.write('output = $(logdir)$(JOBNAME).$(Cluster).out'+'\n')
    s.write('error = $(logdir)$(JOBNAME).$(Cluster).err'+'\n')
    s.write('log = $(logdir)$(JOBNAME).$(Cluster).log'+'\n')
    s.write('\n')
    s.write('notification   = never'+'\n')
    s.write('universe       = vanilla'+'\n')
    s.write('\n')
    s.write('should_transfer_files = YES'+'\n')
    s.write('request_memory = 8GB'+'\n')
    s.write('getenv = True'+'\n')

    s.write('\n')
    if SignalSub:
        # s.write('arguments = -m $(mass) -c $(channel) -p $(profile) --process $(proc) --mcinj $(mcinj) --mcfit $(mcfit)  -n $(Ntrials) -f $(file) -l --null --GPmodel $(gpmodel) --GPinject $(gpinject)'+'\n')
        s.write('arguments = -m $(mass) -c $(channel) -p $(profile) --process $(proc) --mcinj $(mcinj) --mcfit $(mcfit)  -n $(Ntrials) -f $(file) -l --GPmodel $(gpmodel) --GPinject $(gpinject)'+'\n')
    else:
        s.write('arguments = -m $(mass) -c $(channel) -p $(profile) --process $(proc) --mcinj $(mcinj) --mcfit $(mcfit)  -n $(Ntrials) -f $(file) --null --GPmodel $(gpmodel) --GPinject $(gpinject)'+'\n')

    s.write('queue')
    s.close()    

In [3]:
submit = '/home/tchau/code/condor_submit/TSdist/submit.sh'
path = '/home/tchau/code/condor_submit/TSdist/{}_{}'.format(channel, mass)
bkgtrial = 0
if not (os.path.exists(path)): os.makedirs(path)
with open(submit, 'w') as s:
    s.write("#!/bin/sh\n")
    for j in range(Njobs):
    # for j in range(10):  
        dag_file = '{}/TSdist_job{}_bkgtrial{}_SignalSub{}_{}_{}_{}.dag'.format(path, j, bkgtrial, SignalSub, channel, mass, profile)
        with open(dag_file, 'w') as f:
            if SignalSub:
                f.write('JOB TSdist_job{}_bkgtrial{}_SignalSub{}_{}_{}_{}.dag'.format(j, bkgtrial, SignalSub, channel, mass, profile)+' TSdist_SignalSubLLH.submit \n')
            else:
                f.write('JOB TSdist_job{}_bkgtrial{}_SignalSub{}_{}_{}_{}.dag'.format(j, bkgtrial, SignalSub, channel, mass, profile)+' TSdist_NormalLLH.submit \n')
   
            f.write('VARS TSdist_job{}_bkgtrial{}_SignalSub{}_{}_{}_{}.dag'.format(j, bkgtrial, SignalSub, channel, mass, profile)+' JOBNAME="TSdist_job{}_bkgtrial{}_SignalSub{}_{}_{}_{}.dag"'.format(j, bkgtrial, SignalSub, channel, mass, profile) +' mass="{}" channel="{}" profile="{}" set="{}" Ntrials="{}" file="{}" bkgtrial="{}"'.format(mass, channel, profile, set, Ntrials, j, bkgtrial)+'\n')
        f.close()
        s.write("condor_submit_dag -config dagman.config {}\n".format(dag_file))
        print("condor_submit_dag -config dagman.config {}\n".format(dag_file))
s.close()

condor_submit_dag -config dagman.config /home/tchau/code/condor_submit/TSdist/WW_5000/TSdist_job0_bkgtrial0_SignalSubTrue_WW_5000_NFW.dag

condor_submit_dag -config dagman.config /home/tchau/code/condor_submit/TSdist/WW_5000/TSdist_job1_bkgtrial0_SignalSubTrue_WW_5000_NFW.dag

condor_submit_dag -config dagman.config /home/tchau/code/condor_submit/TSdist/WW_5000/TSdist_job2_bkgtrial0_SignalSubTrue_WW_5000_NFW.dag

condor_submit_dag -config dagman.config /home/tchau/code/condor_submit/TSdist/WW_5000/TSdist_job3_bkgtrial0_SignalSubTrue_WW_5000_NFW.dag

condor_submit_dag -config dagman.config /home/tchau/code/condor_submit/TSdist/WW_5000/TSdist_job4_bkgtrial0_SignalSubTrue_WW_5000_NFW.dag



In [6]:
submit = '/home/tchau/code/condor_submit/TSdist/submit.sh'
path = '/home/tchau/code/condor_submit/TSdist/{}_{}'.format(channel, mass)
if not (os.path.exists(path)): os.makedirs(path)

gpinjs = [None, 'pi0', 'pi0_IC', 'KRA50', 'KRA50_IC']
# gpmodels = ['pi0', 'pi0_IC', 'KRA50', 'KRA50_IC']
gpmodels = [None, 'pi0', 'pi0_IC', 'KRA50', 'KRA50_IC']

bkgtrial = 0

gpinjs = [None]
gpmodels = [None]

with open(submit, 'w') as s:
    s.write("#!/bin/sh\n")
    for gpinj in gpinjs:
        for gpmodel in gpmodels:
            for j in range(Njobs):
            # for j in range(10):  
                dag_file = f'{path}/TSdist_job{j}_SignalSub{SignalSub}_{channel}_{mass}_{profile}_{process}_mcinj{mcinj}_mcfit{mcfit}_gpinj{gpinj}_gpmodel{gpmodel}.dag'
                with open(dag_file, 'w') as f:
                    if SignalSub:
                        f.write(f'JOB TSdist_job{j}_SignalSub{SignalSub}_{channel}_{mass}_{profile}_{process}_mcinj{mcinj}_mcfit{mcfit}_gpinj{gpinj}_gpmodel{gpmodel}' +' TSdist_SignalSubLLH.submit \n')
                    else:
                        f.write(f'JOB TSdist_job{j}_SignalSub{SignalSub}_{channel}_{mass}_{profile}_{process}_mcinj{mcinj}_mcfit{mcfit}_gpinj{gpinj}_gpmodel{gpmodel}'+' TSdist_NormalLLH.submit \n')
        
                    f.write(f'VARS TSdist_job{j}_SignalSub{SignalSub}_{channel}_{mass}_{profile}_{process}_mcinj{mcinj}_mcfit{mcfit}_gpinj{gpinj}_gpmodel{gpmodel}'+
                            f' JOBNAME="TSdist_job{j}_SignalSub{SignalSub}_{channel}_{mass}_{profile}_{process}_mcinj{mcinj}_mcfit{mcfit}_gpinj{gpinj}_gpmodel{gpmodel}"' 
                            +' mass="{}" channel="{}" profile="{}" proc="{}" mcinj="{}" mcfit="{}" Ntrials="{}" file="{}" gpmodel="{}" gpinject="{}" '.format(mass, channel, profile, process, mcinj, mcfit, Ntrials, j, gpmodel, gpinj)+'\n')
                f.close()
                s.write("condor_submit_dag -config dagman.config {}\n".format(dag_file))
                print("condor_submit_dag -config dagman.config {}\n".format(dag_file))
s.close()

condor_submit_dag -config dagman.config /home/tchau/code/condor_submit/TSdist/WW_5000/TSdist_job0_SignalSubTrue_WW_5000_NFW_ann_mcinj1122_disCSMSx-3_mcfit1122_gpinjNone_gpmodelNone.dag

condor_submit_dag -config dagman.config /home/tchau/code/condor_submit/TSdist/WW_5000/TSdist_job1_SignalSubTrue_WW_5000_NFW_ann_mcinj1122_disCSMSx-3_mcfit1122_gpinjNone_gpmodelNone.dag

condor_submit_dag -config dagman.config /home/tchau/code/condor_submit/TSdist/WW_5000/TSdist_job2_SignalSubTrue_WW_5000_NFW_ann_mcinj1122_disCSMSx-3_mcfit1122_gpinjNone_gpmodelNone.dag

condor_submit_dag -config dagman.config /home/tchau/code/condor_submit/TSdist/WW_5000/TSdist_job3_SignalSubTrue_WW_5000_NFW_ann_mcinj1122_disCSMSx-3_mcfit1122_gpinjNone_gpmodelNone.dag

condor_submit_dag -config dagman.config /home/tchau/code/condor_submit/TSdist/WW_5000/TSdist_job4_SignalSubTrue_WW_5000_NFW_ann_mcinj1122_disCSMSx-3_mcfit1122_gpinjNone_gpmodelNone.dag

